In [1]:
import os
from dotenv import load_dotenv, find_dotenv
from itertools import chain, starmap
import json
import pandas as pd
import weaviate
from queries import search_datasets_query, browse_datasets_query

# Pandas display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

In [2]:
client = weaviate.Client(os.environ["WEAVIATE_ENDPOINT"])

In [3]:
def flatten_json(dictionary, sep='.'):
    """Flatten a nested json file. For a list of dictionaries, use this
    inside a for loop before converting to pandas DataFrame."""

    def unpack(parent_key, parent_value):
        """Unpack one level of nesting in json file"""
        # Unpack one level only!!!
        
        if isinstance(parent_value, dict):
            for key, value in parent_value.items():
                temp1 = parent_key + sep + key
                yield temp1, value
        elif isinstance(parent_value, list):
            i = 0 
            for value in parent_value:
                temp2 = parent_key + sep +str(i) 
                i += 1
                yield temp2, value
        else:
            yield parent_key, parent_value    


    # Keep iterating until the termination condition is satisfied
    while True:
        # Keep unpacking the json file until all values are atomic elements (not dictionary or list)
        dictionary = dict(chain.from_iterable(starmap(unpack, dictionary.items())))
        # Terminate condition: not any value in the json file is dictionary or list
        if not any(isinstance(value, dict) for value in dictionary.values()) and \
           not any(isinstance(value, list) for value in dictionary.values()):
            break

    return dictionary

## Semantic Search

In [7]:
def run_semantic_search_query(concepts, limit=1000, distance=0.8, search_datasets_query=search_datasets_query):

    search_datasets_query = search_datasets_query.format(
        concepts=json.dumps(concepts.split(",")), 
        limit=str(limit),
        distance=str(distance)
        )

    return client.query.raw(search_datasets_query)['data']['Get']['Dataset']

In [9]:
# search_datasets_query = search_datasets_query.format(
#     concepts=json.dumps("k2".split(",")), 
#     limit=str(1000),
#     distance=str(0.8)
#     )

# res = client.query.raw(search_datasets_query)#['data']['Get']['Dataset']

In [10]:
data = run_semantic_search_query("k2")

In [11]:
data

[{'_additional': {'distance': 0.44141692},
  'description': 'The K2 mission observed 100 square degrees for 80 days each across 20 different pointings along the ecliptic, collecting high-precision photometry for a selection of targets within each field. The mission began when the original Kepler mission ended due to loss of the second reaction wheel in 2011. More information about the K2 mission is available at [MAST](https://archive.stsci.edu/k2/).\n',
  'documentation': 'http://astroquery.readthedocs.io/en/latest/mast/mast.html',
  'hasPublication': None,
  'hasResource': [{'arn': 'arn:aws:s3:::stpubdata/k2',
    'description': 'K2 Mission data files',
    'region': 'us-east-1',
    'requesterPays': False,
    'type': 'S3 Bucket'},
   {'arn': 'arn:aws:sns:us-east-1:879230861493:stpubdata',
    'description': 'Notifications for new data',
    'region': 'us-east-1',
    'requesterPays': None,
    'type': 'SNS Topic'}],
  'hasToolOrApplication': None,
  'hasTutorial': None,
  'managedBy

In [13]:
flat_data = [flatten_json(item) for item in data]

In [14]:
df = pd.DataFrame(flat_data)

In [15]:
df.head()

,_additional.distance,description,documentation,hasPublication,hasResource.0.arn,hasResource.0.description,hasResource.0.region,hasResource.0.requesterPays,hasResource.0.type,hasResource.1.arn,hasResource.1.description,hasResource.1.region,hasResource.1.requesterPays,hasResource.1.type,hasToolOrApplication,hasTutorial,managedBy.0.name,name,tags,hasPublication.0.authorName,hasPublication.0.title,hasPublication.0.url,hasResource.2.arn,hasResource.2.description,hasResource.2.region,hasResource.2.requesterPays,hasResource.2.type,hasResource.3.arn,hasResource.3.description,hasResource.3.region,hasResource.3.requesterPays,hasResource.3.type,hasToolOrApplication.0.authorName,hasToolOrApplication.0.title,hasToolOrApplication.0.url,hasToolOrApplication.1.authorName,hasToolOrApplication.1.title,hasToolOrApplication.1.url,hasToolOrApplication.2.authorName,hasToolOrApplication.2.title,hasToolOrApplication.2.url,hasToolOrApplication.3.authorName,hasToolOrApplication.3.title,hasToolOrApplication.3.url,hasToolOrApplication.4.authorName,hasToolOrApplication.4.title,hasToolOrApplication.4.url,hasToolOrApplication.5.authorName,hasToolOrApplication.5.title,hasToolOrApplication.5.url,hasTutorial.0.services,hasTutorial.0.title,hasTutorial.0.url,hasTutorial.1.services,hasTutorial.1.title,hasTutorial.1.url,hasTutorial.2.services,hasTutorial.2.title,hasTutorial.2.url,hasToolOrApplication.6.authorName,hasToolOrApplication.6.title,hasToolOrApplication.6.url,hasPublication.1.authorName,hasPublication.1.title,hasPublication.1.url,hasPublication.2.authorName,hasPublication.2.title,hasPublication.2.url,hasTutorial.3.services,hasTutorial.3.title,hasTutorial.3.url,hasTutorial.4.services,hasTutorial.4.title,hasTutorial.4.url,hasTutorial.5.services,hasTutorial.5.title,hasTutorial.5.url,hasTutorial.6.services,hasTutorial.6.title,hasTutorial.6.url,hasTutorial.7.services,hasTutorial.7.title,hasTutorial.7.url,hasTutorial.8.services,hasTutorial.8.title,hasTutorial.8.url,hasPublication.3.authorName,hasPublication.3.title,hasPublication.3.url
0,0.441417,"The K2 mission observed 100 square degrees for 80 days each across 20 different pointings along the ecliptic, collecting high-precision photometry for a selection of targets within each field. The mission began when the original Kepler mission ended due to loss of the second reaction wheel in 2011. More information about the K2 mission is available at [MAST](https://archive.stsci.edu/k2/).\n",http://astroquery.readthedocs.io/en/latest/mast/mast.html,NaN,arn:aws:s3:::stpubdata/k2,K2 Mission data files,us-east-1,False,S3 Bucket,arn:aws:sns:us-east-1:879230861493:stpubdata,Notifications for new data,us-east-1,None,SNS Topic,NaN,NaN,[Space Telescope Science Institute](http://www.stsci.edu/),K2 Mission Data,"astronomy,aws-pds",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.550664,"The Geo-KOMPSAT-2A (GK2A) is the new generation geostationary meteorological satellite (located in 128.2°E) of the Korea Meteorological Administration (KMA). The main mission of the GK2A is to observe the atmospheric phenomena over the Asia-Pacific region. The Advance Meteorological Imager (AMI) on GK2A scan the Earth full disk every 10 minutes and the Korean Peninsula area every 2 minutes with a high spatial resolution of 4 visible channels and 12 infrared channels. In addition, the AMI has an ability of flexible target area scanning useful for monitoring severe weather events such as typhoon and volcanic eruption and so on. And for space weather mission, the Korea Space wEather Monitor (KSEM) on the GK2A observes the space environment with the particle detector, magnetometer and charging monitor. For questions regarding GK2A imagery specifications, visit the GK2A site at https://nmsc.kma.go.kr/enhome/html/base/

## Browse Datasets

In [10]:
print(browse_datasets_query.format())


{
    Get {
        Dataset (
            limit: 1000
        ) {
            name
            description
            documentation
            tags
            managedBy {
                ... on Publisher {
                    name
                }
            }
            hasResource {
                ... on Resource {
                    arn
                    region
                    description
                    type
                    requesterPays
                }
            }
            hasTutorial {
                ... on Tutorial {
                    title
                    url
                    services
                }
            }
            hasPublication {
                ... on Publication {
                    title
                    url
                    authorName
                }
            }
            hasToolOrApplication {
                ... on ToolOrApplication {
                    title
                    url
                    a

In [4]:
res

{'data': {'Get': {'Dataset': [{'description': 'The Virginia Coastal Resilience Master Plan builds on the 2020 Virginia Coastal Resilience Master Planning Framework, which outlined the goals and principles of the Commonwealth’s statewide coastal resilience strategy. Recognizing the urgent challenge flooding already poses, the Commonwealth developed Phase One of the Master Plan on an accelerated timeline and focused this first assessment on the impacts of tidal and storm surge coastal flooding on coastal Virginia.  The Master Plan leveraged the combined efforts of more than two thousand stakeholders, subject matter experts, and government personnel. We centered the development of this plan around three core components:\n<br/>\n<br/>\nA Technical Study compiled essential data, research, processes, products, and resilience efforts in the Coastal Resilience Database, which forms much of basis of this plan and the Coastal Resilience Web Explorer;\n<br/>\n<br/>\nA Technical Advisory Committee

In [5]:
def run_browse_datasets_query():
    return client.query.raw(browse_datasets_query.format())['data']['Get']['Dataset']

In [6]:
datasets = run_browse_datasets_query()

In [7]:
# sort by name
datasets.sort(key=lambda x: x['name'])

In [8]:
len(datasets)

453